In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-cloud-sql-mysql-python/blob/main/samples/langchain_quick_start.ipynb)

---
# **Introduction**

In this codelab, you'll learn how to create a powerful interactive generative AI application using Retrieval Augmented Generation powered by [Cloud SQL for MySQL](https://cloud.google.com/sql/docs/mysql) and [LangChain](https://www.langchain.com/). We will be creating an application grounded in a [Netflix Movie dataset](https://www.kaggle.com/datasets/shivamb/netflix-shows), allowing you to interact with movie data in exciting new ways.

## Prerequisites

* A basic understanding of the Google Cloud Console
* Basic skills in command line interface and Google Cloud shell
* Basic Python knowledge

## What you'll learn

* How to deploy a Cloud SQL for MySQL instance
* How to use Cloud SQL for MySQL as a DocumentLoader
* How to use Cloud SQL for MySQL as a VectorStore
* How to use Cloud SQL for MySQL for ChatMessageHistory storage

## What you'll need
* A Google Cloud Account and Google Cloud Project
* A web browser such as [Chrome](https://www.google.com/chrome/)

# **Setup and Requirements**

In the following instructions you will learn to:
1. Install required dependencies for our application
2. Set up authentication for our project
3. Set up a Cloud SQL for MySQL Instance
4. Import the data used by our application

## Install dependencies
First you will need to install the dependencies needed to run this demo app.

In [ ]:
%pip install --upgrade --quiet langchain-google-cloud-sql-mysql langchain-google-vertexai langchain

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Authenticate to Google Cloud within Colab
In order to access your Google Cloud Project from this notebook, you will need to Authenticate as an IAM user.

In [ ]:
from google.colab import auth

auth.authenticate_user()

## Connect Your Google Cloud Project
Time to connect your Google Cloud Project to this notebook so that you can leverage Google Cloud from within Colab.

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
PROJECT_ID = ""  # @param {type:"string"}

# Quick input validations.
assert PROJECT_ID, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {PROJECT_ID}

## Configure Your Google Cloud Project

Configure the following in your Google Cloud Project.

1. IAM principal (user, service account, etc.) with the [Cloud SQL Client][client-role] role. The user logged into this notebook will be used as the IAM principal and will be granted the Cloud SQL Client role.

[client-role]: https://cloud.google.com/sql/docs/mysql/roles-and-permissions

In [ ]:
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

2. Enable the APIs for Cloud SQL and Vertex AI within your project.

In [ ]:
# Enable GCP services
!gcloud services enable sqladmin.googleapis.com aiplatform.googleapis.com

## Set up Cloud SQL
You will need a **MySQL** Cloud SQL instance for the following stages of this notebook.

### Create a MySQL Instance
Running the below cell will verify the existence of the Cloud SQL instance and or create a new instance if one does not exist.

A database named `langchain_db` will be created and used for the rest of the quickstart.

**Note:** MySQL vector support is only available on MySQL instances with version **>= 8.0.36**.

> For existing instances, you may need to perform a [self-service maintenance update](MYSQL_8_0_36.R20240401.03_00) to update your maintenance version to **MYSQL_8_0_36.R20240401.03_00** or greater. Once updated, [configure your database flags](https://cloud.google.com/sql/docs/mysql/flags) to have thew new **cloudsql_vector** flag to "On".

> ⏳ - Creating a Cloud SQL instance may take a few minutes.

In [ ]:
#@markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance. Once filled in, run the cell.

# Please fill in these values.
REGION = "us-central1" #@param {type:"string"}
INSTANCE_NAME = "langchain-quickstart-instance" #@param {type:"string"}
DATABASE_NAME = "langchain_db"
PASSWORD = input("Please provide a password to be used for 'root' database user: ")

# Quick input validations.
assert REGION, "⚠️ Please provide a Google Cloud region"
assert INSTANCE_NAME, "⚠️ Please provide the name of your instance"
assert DATABASE_NAME, "⚠️ Please provide the name of your database"

# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {INSTANCE_NAME} --format="value(databaseVersion)"
if database_version[0].startswith("MYSQL"):
  print("Found existing MySQL Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  !gcloud sql instances create {INSTANCE_NAME} --database-version=MYSQL_8_0_36 \
    --region={REGION} --cpu=1 --memory=4GB --root-password={PASSWORD} \
    --database-flags=cloudsql_iam_authentication=On,cloudsql_vector=On

databases = !gcloud sql databases list --instance={INSTANCE_NAME} --format="value(name)"
if DATABASE_NAME not in databases:
  print("Creating 'langchain_db' database for the quickstart...")
  !gcloud sql databases create {DATABASE_NAME} --instance={INSTANCE_NAME}

## Import data to your database

Now that you have your database, you will need to import data! We will be using a [Netflix Dataset from Kaggle](https://www.kaggle.com/datasets/shivamb/netflix-shows). Here is what the data looks like:

| show_id | type    | title                | director         | cast                                                                                                                                                  | country       | date_added        | release_year | rating | duration  | listed_in                                    | description                                                                                                                                                                           |
|---------|---------|----------------------|------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|---------------|-------------------|--------------|--------|-----------|----------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| s1      | Movie   | Dick Johnson Is Dead | Kirsten Johnson  |                                                                                                                                                        | United States | September 25, 2021 | 2020         | PG-13  | 90 min    | Documentaries                                | As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.                              |
| s2      | TV Show | Blood & Water        |                  | Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng | South Africa  | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, TV Dramas, TV Mysteries | After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.                                   |
| s3      | TV Show | Ganglands            | Julien Leclercq  | Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera                                   |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Crime TV Shows, International TV Shows, TV Action & Adventure | To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.                                     |
| s4      | TV Show | Jailbirds New Orleans |                  |                                                                                                                                                        |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Docuseries, Reality TV                        | Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.                                   |
| s5      | TV Show | Kota Factory         |                  | Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar                                                 | India        | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, Romantic TV Shows, TV Comedies | In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life. |


You won't need to directly load the csv data into your database. Instead we prepared a table, "netflix_titles", in the format of a `.sql` file for MySQL. You can easily import the table into your database with one `gcloud` command.

In [ ]:
# Import the Netflix titles table using gcloud command
import_command_output = !gcloud sql import sql {INSTANCE_NAME} gs://cloud-samples-data/langchain/cloud-sql/mysql/netflix_titles.sql --database={DATABASE_NAME} --quiet

if "Imported data" in str(import_command_output):
    print(import_command_output)
elif "already exists" in str(import_command_output):
    print("Did not import because the table already existed.")
else:
    raise Exception(f"The import seems to have failed:\n{import_command_output}")

# **Use case 1: Cloud SQL for MySQL as a document loader**

Now that you have data in your database, you are ready to use Cloud SQL for MySQL as a [document loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/). This means we will pull data from the database and load it into memory as documents. We can then feed these documents into the vector store.

Next let's connect to our Cloud SQL MySQL instance using the `MySQLEngine` class from the `langchain-google-cloud-sql-mysql` package.

In [ ]:
from langchain_google_cloud_sql_mysql import MySQLEngine

mysql_engine = MySQLEngine.from_instance(
    project_id=PROJECT_ID,
    instance=INSTANCE_NAME,
    region=REGION,
    database=DATABASE_NAME,
    user="root",
    password=PASSWORD,
)

Once we initialize a MySQLEngine object, we can pass it into the `MySQLLoader` to connect to a specific database. As you can see we also pass in a query, table_name and a list of columns. The query tells the loader what query to use to pull data. The "content_columns" argument refers to the columns that will be used as "content" in the document object we will later construct. The rest of the columns in that table will become the "metadata" associated with the documents.

In [ ]:
from langchain_google_cloud_sql_mysql import MySQLLoader

table_name = "netflix_titles"
content_columns = ["title", "director", "cast", "description"]
loader = MySQLLoader(
    engine=mysql_engine,
    query=f"SELECT * FROM `{table_name}`;",
    content_columns=content_columns,
)

Then let's load our documents from our database using our document loader. You can see the first 5 documents from the database here. Nice, you just used Cloud SQL for MySQL as a LangChain document loader!

In [ ]:
documents = loader.load()
print(f"Loaded {len(documents)} documents from the database. \n5 Examples:")
for doc in documents[:5]:
    print(doc)

Loaded 8807 documents from the database. 
5 Examples:
page_content='Dick Johnson Is Dead Kirsten Johnson  As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.' metadata={'show_id': 's1', 'type': 'Movie', 'country': 'United States', 'date_added': 'September 25, 2021', 'release_year': 2020, 'rating': 'PG-13', 'duration': '90 min', 'listed_in': 'Documentaries'}
page_content='Blood & Water  Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.' metadata={'show_id': 's2', 'type': 'TV Show', 'country': 

# **Use case 2: Cloud SQL for MySQL as a vector store**

---



Now, let's learn how to put all of the documents we just loaded into a [vector store](https://python.langchain.com/docs/modules/data_connection/vectorstores/) so that we can use vector search to answer our user's questions!

### Create Your Vector Store table

Based on the documents that we loaded before, we want to create a table with a vector column as our vector store. We will start it by intializing a vector table by calling the `init_vectorstore_table` function from our `engine`. As you can see we list all of the columns for our metadata. We also specify a vector size, 768, that corresponds with the length of the vectors computed by the model our embeddings service uses, Vertex AI's textembedding-gecko.


In [ ]:
from langchain_google_cloud_sql_mysql import Column

vector_table_name = "vector_netflix_titles"

mysql_engine.init_vectorstore_table(
    table_name=vector_table_name,
    vector_size=768,
    metadata_columns=[
        Column("show_id", "VARCHAR(50)", nullable=True),
        Column("type", "VARCHAR(50)", nullable=True),
        Column("country", "VARCHAR(50)", nullable=True),
        Column("date_added", "VARCHAR(50)", nullable=True),
        Column("release_year", "INTEGER", nullable=True),
        Column("rating", "VARCHAR(50)", nullable=True),
        Column("duration", "VARCHAR(50)", nullable=True),
        Column("listed_in", "VARCHAR(50)", nullable=True),
    ],
    overwrite_existing=True,  # Enabling this will recreate the table if exists.
)

### Try inserting the documents into the vector table

Now we will create a vector_store object backed by our vector table in the Cloud SQL database. Let's load the data from the documents to the vector table. Note that for each row, the embedding service will be called to compute the embeddings to store in the vector table.

Pricing details can be found [here](https://cloud.google.com/vertex-ai/pricing).

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_mysql import MySQLVectorStore

# Initialize the embedding service. In this case we are using version 003 of Vertex AI's textembedding-gecko model.
# In general, it is good practice to specify the model version used.
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@003", project=PROJECT_ID
)

vector_store = MySQLVectorStore(
    engine=mysql_engine,
    embedding_service=embeddings_service,
    table_name=vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

Now let's try to put the documents data into the vector table. Here is a code example to load the first 5 documents in the list.

In [ ]:
import uuid

docs_to_load = documents[:5]

# ! Uncomment the following line to load all 8,800+ documents to the database vector table with calling the embedding service.
# docs_to_load = documents

ids = [str(uuid.uuid4()) for i in range(len(docs_to_load))]
vector_store.add_documents(docs_to_load, ids)

### Import the rest of your data into your vector table

You don't have to call the embedding service 8,800 times to load all the documents for the demo. Instead, we have prepared a table with the all 8,800+ rows with pre-computed embeddings in a `.sql` file. Again, let's import to our DB using `gcloud` command.

It will restore the `.sql` file to a table with vectors called `vector_netflix_titles`.

In [ ]:
# Import the netflix titles with vector table using gcloud command
import_command_output = !gcloud sql import sql {INSTANCE_NAME} gs://cloud-samples-data/langchain/cloud-sql/mysql/vector_netflix_titles.sql --database={DATABASE_NAME} --quiet

if "Imported data" in str(import_command_output):
    print(import_command_output)
elif "already exists" in str(import_command_output):
    print("Did not import because the table already existed.")
else:
    raise Exception(f"The import seems failed:\n{import_command_output}")

# **Use case 3: Cloud SQL for MySQL as Chat Memory**

Next we will add chat history (called [“memory” in the context of LangChain](https://python.langchain.com/docs/modules/memory/)) to our application so the LLM can retain context and information across multiple interactions, leading to more coherent and sophisticated conversations or text generation. We can use Cloud SQL for MySQL as “memory” storage in our application so that the LLM can use context from prior conversations to better answer the user’s prompts. First let's initialize Cloud SQL for MySQL as memory storage.

In [ ]:
from langchain_google_cloud_sql_mysql import MySQLChatMessageHistory

message_table_name = "message_store"

mysql_engine.init_chat_history_table(table_name=message_table_name)

chat_history = MySQLChatMessageHistory(
    mysql_engine,
    session_id="my-test-session",
    table_name=message_table_name,
)

Here is an example of how you would add a user message and how you would add an ai message.

In [ ]:
chat_history.add_user_message("Hi!")
chat_history.add_ai_message("Hello there. I'm a model and am happy to help!")

chat_history.messages

[HumanMessage(content='Hi!'),
 AIMessage(content="Hello there. I'm a model and am happy to help!")]

# **Conversational RAG Chain backed by Cloud SQL for MySQL**

So far we've tested with using Cloud SQL for MySQL as document loader, vector store and chat memory. Now let's put it all together with a `ConversationalRetrievalChain`.

We will build a chat bot that can answer movie related questions based on the vector search results.

First let's initialize all of our MySQL engine object to use as a connection in our vector store and chat_history.

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings, VertexAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_google_cloud_sql_mysql import (
    MySQLChatMessageHistory,
    MySQLEngine,
    MySQLVectorStore,
)

# Initialize the embedding service
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

# Initialize the engine
mysql_engine = MySQLEngine.from_instance(
    project_id=PROJECT_ID,
    instance=INSTANCE_NAME,
    region=REGION,
    database=DATABASE_NAME,
    user="root",
    password=PASSWORD,
)

# Initialize the Vector Store
vector_table_name = "vector_netflix_titles"
vector_store = MySQLVectorStore(
    engine=mysql_engine,
    embedding_service=embeddings_service,
    table_name=vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

# Initialize the MySQLChatMessageHistory
chat_history = MySQLChatMessageHistory(
    mysql_engine,
    session_id="my-test-session",
    table_name="message_store",
)

Let's create a prompt for the LLM. Here we can add instructions specific to our application, such as "Don't make things up".

In [ ]:
# Prepare some prompt templates for the ConversationalRetrievalChain
prompt = PromptTemplate(
    template="""Use all the information from the context and the conversation history to answer new question. If you see the answer in previous conversation history or the context. \
Answer it with clarifying the source information. If you don't see it in the context or the chat history, just say you \
didn't find the answer in the given data. Don't make things up.

Previous conversation history from the questioner. "Human" was the user who's asking the new question. "Assistant" was you as the assistant:
```{chat_history}
```

Vector search result of the new question:
```{context}
```

New Question:
```{question}```

Answer:""",
    input_variables=["context", "question", "chat_history"],
)
condense_question_prompt_passthrough = PromptTemplate(
    template="""Repeat the following question:
{question}
""",
    input_variables=["question"],
)

Now let's use our vector store as a retreiver. Retreiver's in Langchain allow us to literally "retrieve" documents.

In [ ]:
# Initialize retriever, llm and memory for the chain
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.8}
)

Now let's initialize our LLM, in this case we are using Vertex AI's "gemini-pro".

In [ ]:
llm = VertexAI(model_name="gemini-pro", project=PROJECT_ID)

We clear our chat history, so that our application starts without any prior context to other conversations we have had with the application.

In [ ]:
chat_history.clear()

memory = ConversationSummaryBufferMemory(
    llm=llm,
    chat_memory=chat_history,
    output_key="answer",
    memory_key="chat_history",
    return_messages=True,
)

Now let's create a conversational retrieval chain. This will allow the LLM to use chat history in it's responses, meaning we can ask it follow up questions to our questions instead of having to start from scratch after each inquiry.

In [ ]:
# create the ConversationalRetrievalChain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    verbose=False,
    memory=memory,
    condense_question_prompt=condense_question_prompt_passthrough,
    combine_docs_chain_kwargs={"prompt": prompt},
)

# ask some questions
q = "What movie was Brad Pitt in?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "How about Johnny Depp?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "Are there movies about animals?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

# browser the chat history
chat_history.messages

Question: What movie was Brad Pitt in?
Answer: Brad Pitt was in the following movies: Inglourious Basterds, By the Sea, Killing Them Softly, Babel, War Machine

Question: How about Johnny Depp?
Answer: Johnny Depp was in the following movies: The Rum Diary, Charlie and the Chocolate Factory, The Tourist, The Imaginarium of Doctor Parnassus, What's Eating Gilbert Grape. (Source: vector search result)

Question: Are there movies about animals?
Answer: Yes, from the vector search result, there are the following movies that feature animals:

- Animals on the Loose: A You vs. Wild Movie
- Rango
- Kung Fu Panda: Secrets of the Scroll
- Balto
- Cats & Dogs: The Revenge of Kitty Galore



[HumanMessage(content='What movie was Brad Pitt in?'),
 AIMessage(content='Brad Pitt was in the following movies: Inglourious Basterds, By the Sea, Killing Them Softly, Babel, War Machine'),
 HumanMessage(content='How about Johnny Depp?'),
 AIMessage(content="Johnny Depp was in the following movies: The Rum Diary, Charlie and the Chocolate Factory, The Tourist, The Imaginarium of Doctor Parnassus, What's Eating Gilbert Grape. (Source: vector search result)"),
 HumanMessage(content='Are there movies about animals?'),
 AIMessage(content='Yes, from the vector search result, there are the following movies that feature animals:\n\n- Animals on the Loose: A You vs. Wild Movie\n- Rango\n- Kung Fu Panda: Secrets of the Scroll\n- Balto\n- Cats & Dogs: The Revenge of Kitty Galore')]